In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
import glob
import librosa
import os
from torch.utils.data import Dataset
from torchvision import datasets

In [ ]:
!pip install speechbrain

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 496 kB 4.3 MB/s 
     |████████████████████████████████| 1.2 MB 52.8 MB/s 
     |████████████████████████████████| 101 kB 12.2 MB/s 
     |████████████████████████████████| 596 kB 70.5 MB/s 
     |████████████████████████████████| 109 kB 52.0 MB/s 
     |████████████████████████████████| 546 kB 54.4 MB/s 
  Created wheel for hyperpyyaml: filename=HyperPyYAML-1.0.1-py3-none-any.whl size=15192 sha256=5fd318626ebef88f1e8315dad1a51e2611f1814802980f3cca3e7e22c1182a8d
  Stored in directory: /root/.cache/pip/wheels/6c/87/65/266d722c3932f81f16332ce842e972be8421e3a9cd3771766b
Successfully built hyperpyyaml
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13


In [ ]:
import torchaudio
from speechbrain.pretrained import EncoderClassifier
import numpy as np
from scipy.io import wavfile
import numpy as np
from scipy.io.wavfile import write
import torch

classifier = EncoderClassifier.from_hparams(source="speechbrain/spkrec-ecapa-voxceleb")
# signal, fs =torchaudio.load('/content/drive/MyDrive/thesis/testfile_generated_e2e.wav')
# embeddings = classifier.encode_batch(signal)

Downloading:   0%|          | 0.00/1.92k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/83.3M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.92k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/5.53M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/129k [00:00<?, ?B/s]

In [ ]:
import torch.nn as nn
import torch.nn.functional as F
base_dir = "/content/drive/MyDrive/DataArray/"

In [ ]:
#Save audio embedding to npy file
import glob
import numpy as np
import matplotlib.pyplot as plt
path = '/content/drive/MyDrive/WAV_VAD/*.wav'
def get_all_filename_from_folder(path):
  return glob.glob(path)
base_dir = "/content/drive/MyDrive/DataArray/"
x = get_all_filename_from_folder(path)
min_mat = []
min_path = []
for _ in range(len(x)):
  frequency, signal = wavfile.read(x[_])
  audio_slice = signal
  audio_slice = audio_slice.reshape(1,-1)
  audio_slice = torch.tensor(audio_slice)
  audio_slice = classifier.encode_batch(audio_slice)
  audio_slice = audio_slice.squeeze()
  audio_slice = np.array(audio_slice)
  name = x[_].split("/")
  name[5] = name[5].split(".")
  name = base_dir + str(name[5][0]) + ".npy"
  np.save(name, audio_slice)




In [3]:
import pandas as pd
import glob
import librosa
import os
from torch.utils.data import Dataset
from torchvision import datasets
import torchaudio
from scipy.io import wavfile
import numpy as np
from scipy.io.wavfile import write
import torch
import torch.nn as nn
import torch.nn.functional as F
import pandas as pd
base_dir = "/content/drive/MyDrive/DataArray/"


In [4]:
data = pd.read_csv("/content/drive/MyDrive/Viet1.csv")
for i in range(len(data)):
  data['Path'][i] = (base_dir + data['Path'][i].split("/")[5].split(".")[0] + ".npy")

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until


In [5]:
from sklearn.model_selection import train_test_split
data_Train, data_Test = train_test_split(data, test_size=0.33, random_state=42)
print(data_Train)

      Unnamed: 0  Unnamed: 0.1    MinCos  \
710          710           710  0.351342   
167          167           167  0.081919   
1009        1009          1009  0.268499   
286          286           286  0.311154   
1711        1711          1711  0.167786   
...          ...           ...       ...   
1638        1638          1638  0.047461   
1095        1095          1095  0.239941   
1130        1130          1130 -0.052040   
1294        1294          1294  0.173743   
860          860           860  0.003244   

                                           Path  Label  
710   /content/drive/MyDrive/DataArray/3387.npy      1  
167   /content/drive/MyDrive/DataArray/1592.npy      0  
1009   /content/drive/MyDrive/DataArray/116.npy      1  
286   /content/drive/MyDrive/DataArray/1738.npy      0  
1711  /content/drive/MyDrive/DataArray/1060.npy      1  
...                                         ...    ...  
1638   /content/drive/MyDrive/DataArray/965.npy      0  
1095   /content

In [6]:
X_train = data_Train['Path']
y_train = data_Train['Label']
X_test = data_Test['Path']
y_test = data_Test['Label']
X_train = X_train.tolist()
y_train = y_train.tolist()
X_test = X_test.tolist() 
y_test = y_test.tolist()

In [7]:
for i in range(1399):
  X_train[i] = np.load(str(X_train[i]))
for i in range(2089 - 1399):
  X_test[i] = np.load(str(X_test[i])) 

In [8]:

class Train_Dataset(Dataset):

    def __init__(self):
        # Initialize data, download, etc.
        # read with numpy or pandas
        xy = X_train
        self.n_samples = 1399
        A = X_train
        B = y_train
        # here the first column is the class label, the rest are the features
        self.x_data = torch.tensor(A) # size [n_samples, n_features]
        self.y_data = torch.tensor(B) # size [n_samples, 1]

    # support indexing such that dataset[i] can be used to get i-th sample
    def __getitem__(self, index):
        return self.x_data[index], self.y_data[index]

    # we can call len(dataset) to return the size
    def __len__(self):
        return self.n_samples



class Test_Dataset(Dataset):

    def __init__(self):
        # Initialize data, download, etc.
        # read with numpy or pandas
        xy = X_test
        self.n_samples = 2089-1399
        A = X_test
        B = y_test
        # here the first column is the class label, the rest are the features
        self.x_data = torch.tensor(A) # size [n_samples, n_features]
        self.y_data = torch.tensor(B) # size [n_samples, 1]

    # support indexing such that dataset[i] can be used to get i-th sample
    def __getitem__(self, index):
        return self.x_data[index], self.y_data[index]

    # we can call len(dataset) to return the size
    def __len__(self):
        return self.n_samples

In [9]:
Training_Data = Train_Dataset()

# get first sample and unpack
first_data = Training_Data[0]
features, labels = first_data
print(features, labels)
Testing_Data = Test_Dataset()
first_data = Testing_Data[0]
features, labels = first_data
print(features, labels)

tensor([ 27.6138,  20.1814, -16.6200, -33.8050,  47.2734, -24.1429, -14.0186,
         -9.8726,  -0.9023,  11.9569, -11.4139,  26.6349, -17.5362,  -2.0389,
         -4.7936, -28.9854,  21.3499,  44.7589,  31.0030,  46.6975,  11.6168,
         21.3573,  34.8090,  -5.2635,   2.2842, -14.0599,  17.5967,   1.6669,
         -2.8082,  46.0592, -10.4999, -22.6125,   6.7148,  31.1869,   2.3270,
         21.0034,  -4.4198, -18.0742,   5.9836,  -2.4145, -25.2776,  17.4939,
         22.1291,  -2.8234, -19.6992, -14.6097,  18.8721,  -5.2160,  13.0147,
         19.2362,   3.5250,  10.7447, -15.6887,   2.5021, -18.2138, -22.5547,
         11.1968, -41.5183,  26.6056,  -4.5710,  18.3137,  36.5339,  22.2308,
         20.5667,  13.7780,  15.5876, -50.6113, -33.9438,  11.8355, -23.0709,
        -31.4931,   2.3904,  14.0556,  -1.5613, -12.6702,   8.4356,  21.7077,
        -24.3436,  -6.8926,   0.6452,  19.0446, -25.4615,   3.5348,  13.1019,
         26.4721,   0.5597, -43.7808, -31.2205,   6.9008,   8.49

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:12: UserWarning: Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy.array() before converting to a tensor. (Triggered internally at  ../torch/csrc/utils/tensor_new.cpp:210.)
  if sys.path[0] == '':


In [10]:
X_test = np.array(X_test)

In [11]:
from sklearn.metrics import mean_squared_error, r2_score
import numpy as np

distance_6 = []
from sklearn.neighbors import KNeighborsRegressor
for i in range(20):
    knn_model = KNeighborsRegressor(n_neighbors=i+1, weights='distance').fit(X_train, y_train)
    predicted_values = knn_model.predict(X_test)
    predict_df = pd.DataFrame({"Dependent_Test" : y_test, "Dependent_Predicted" : predicted_values})
    mse_knn_distance = mean_squared_error(predict_df.Dependent_Predicted, predict_df.Dependent_Test)
    print(" k = " + str(i+1) + " Mean Squared Error = ", mean_squared_error(predict_df.Dependent_Predicted, predict_df.Dependent_Test))
    distance_6.append(mean_squared_error(predict_df.Dependent_Predicted, predict_df.Dependent_Test))

 k = 1 Mean Squared Error =  0.2710144927536232
 k = 2 Mean Squared Error =  0.21105622891768339
 k = 3 Mean Squared Error =  0.19427056898183792
 k = 4 Mean Squared Error =  0.17981893170501717
 k = 5 Mean Squared Error =  0.17912943705443357
 k = 6 Mean Squared Error =  0.17914148084099604
 k = 7 Mean Squared Error =  0.17823082882875663
 k = 8 Mean Squared Error =  0.1757074667793442
 k = 9 Mean Squared Error =  0.173968421355182
 k = 10 Mean Squared Error =  0.17212850773573382
 k = 11 Mean Squared Error =  0.17407197397698337
 k = 12 Mean Squared Error =  0.1759921228701635
 k = 13 Mean Squared Error =  0.17728753595789642
 k = 14 Mean Squared Error =  0.17864356842948587
 k = 15 Mean Squared Error =  0.17995164848871115
 k = 16 Mean Squared Error =  0.18280454892586068
 k = 17 Mean Squared Error =  0.18418969178575226
 k = 18 Mean Squared Error =  0.1866707960439735
 k = 19 Mean Squared Error =  0.1881329504485361
 k = 20 Mean Squared Error =  0.18870778106706088


In [12]:
from sklearn.metrics import mean_squared_error, r2_score
import numpy as np

distance_6 = []
from sklearn.neighbors import KNeighborsRegressor
# for i in range(20):
knn_model = KNeighborsRegressor(n_neighbors=10, weights='distance').fit(X_train, y_train)
predicted_values = knn_model.predict(X_test)
predict_df = pd.DataFrame({"Dependent_Test" : y_test, "Dependent_Predicted" : predicted_values})
mse_knn_distance = mean_squared_error(predict_df.Dependent_Predicted, predict_df.Dependent_Test)
    # print(" k = " + str(i+1) + " Mean Squared Error = ", mean_squared_error(predict_df.Dependent_Predicted, predict_df.Dependent_Test))
    # distance_6.append(mean_squared_error(predict_df.Dependent_Predicted, predict_df.Dependent_Test))

In [13]:
predicted_values = knn_model.predict(X_test)
k = 0
for i in range(len(predicted_values)):
  if predicted_values[i]>=0.5:
    predicted_values[i]=1
  else:
    predicted_values[i]=0
for i in range(len(predicted_values)):
  if y_test[i]==predicted_values[i]:
    k = k + 1
  # print("ground truth: " + str(y_test[i]) + "predict: " + str(predicted_values[i]))


print(str(k/len(predicted_values)*100) + "%")

75.36231884057972%


In [28]:
import numpy as np
from sklearn.linear_model import SGDClassifier
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import make_pipeline
k = 0
clf = make_pipeline(StandardScaler(),
                    SGDClassifier(max_iter=10000, tol=1e-3))
clf.fit(X_train, y_train)
predict = clf.predict(X_test)
for i in range(len(predict)):
  if predict[i]==y_test[i]:
    k = k + 1
print(str(k/len(X_test)*100) + "%")

70.86956521739131%
